In [1]:
import numpy as np
import h5py
import time
import logging
import matplotlib.pyplot as plt
import pandas as pd
import random
import os

In [2]:
    def read_data( hf, target_device, house_list):
        """Load data of houses
        """

        if len(house_list) == 0:
            return [], []

        else:
            aggregates = []
            targets = []
            timestamps=[]
        
            for house in house_list:

                aggregate = hf[house]['aggregate'][:]
                target = hf[house][target_device][:]
                timestamp=hf[house]['unix'][:]
                
                aggregates.append(aggregate)
                targets.append(target)
                timestamps.append(timestamp)
            aggregates = np.concatenate(aggregates, axis=0)
            targets = np.concatenate(targets, axis=0)
            times=np.concatenate(timestamps,axis=0)
            
            return aggregates, targets,timestamps

In [3]:
def extract(application,house,m_off,m_on,m_threshold,num):
    load_time = time.time()
    target_device=application
    train_house_list=[house]
    hf = h5py.File('data-raw.h5', 'r')
    (train_x, y,timestamps) = read_data(hf, target_device, train_house_list)
  
   
    index =[i for i in range(0, len(train_y)*10, 10)]
    
    data = train_y
    chunk = pd.Series(data, index=index)
    min_off_duration=m_off
    min_on_duration=m_on
    border=1
    on_power_threshold=m_threshold
    when_on = chunk >= on_power_threshold


    state_changes = when_on.astype(np.int8).diff()
    
    switch_on_events = np.where(state_changes == 1)[0]
    switch_off_events = np.where(state_changes == -1)[0]
    
    del state_changes

    


    if switch_off_events[0] < switch_on_events[0]:
        switch_off_events = switch_off_events[1:]
        
           
    if switch_on_events[-1] > switch_off_events[-1]:
        switch_on_events = switch_on_events[:-1]
        
    assert len(switch_on_events) == len(switch_off_events)
    a=chunk.index[switch_on_events[1:]]
    b=chunk.index[switch_off_events[:-1]]
    c=[]
    d=[]
    for i in range(0,len(a)):
        c.append(a[i])
    off_durations=[]

    for j in range(0,len(b)):
        d.append(b[j])

    if min_off_duration > 0:
        off_durations = np.array(c) - np.array(d)
        above_threshold_off_durations = np.where(
            off_durations >= min_off_duration)[0]
        switch_off_events = switch_off_events[
                np.concatenate([above_threshold_off_durations,
                                [len(switch_off_events)-1]])]
        switch_on_events = switch_on_events[
                np.concatenate([[0], above_threshold_off_durations+1])]
        assert len(switch_on_events) == len(switch_off_events)
  
    activations = []
    n=0
    c=0
    for on, off in zip(switch_on_events, switch_off_events):
        
        duration = (chunk.index[ off] - chunk.index[on])
        if duration < min_on_duration:
            continue
        on -= 1 + border
        if on < 0:
            on = 0
        off += border
        activation = chunk.iloc[on:off]
        if not activation.isnull().values.any() and len(activation)<1070:
            file_path = os.path.join('app_activations/'+str(application)+'/lstm_predict_h'+str(num)+'/dw'+str(c)+'_prediction.npy') 
            dir_path = os.path.dirname(file_path)    
            if not os.path.exists(dir_path):  
                os.makedirs(dir_path)  
            np.save('app_activations/'+str(application)+'/ukdale_predict_h'+str(num)+'/dw'+str(c)+'_prediction.npy', activation)
            c+=1
          

In [ ]:
#kettle 11
ls=["house2","house3","house4","house5","house6","house7","house8","house9","house11","house12","house13","house17","house19","house20","house21"]
num=["2","3","4","5","6","7","8","9","11","12","13","17","19","20","21"]
for h,l in zip(ls,num):
    extract('kettle',h,0,12,2000,l)

In [4]:
ls=["house2"]
num=["2"]
for h,l in zip(ls,num):
    extract('kettle',h,0,12,2000,l)

In [16]:
#microwave 11
ls=["house2","house3","house4","house5","house6","house8","house9","house10","house11","house12","house15","house17","house18","house19","house20"]
num=["2","3","4","5","6","8","9","10","11","12","15","17","18","19","20"]
for h,l in zip(ls,num):
    extract('microwave',h,30,12,200,l)

In [4]:
ls=["house18","house19","house20"]
num=["18","19","20"]
for h,l in zip(ls,num):
    extract('microwave',h,30,12,200,l)

In [42]:
ls=["house18","house19","house20","house21"]
num=["18","19","20","21"]
for h,l in zip(ls,num):
    extract('washingmachine',h,160,1800,20,l)

In [44]:
ls=["house16","house18","house20","house21"]
num=["16","18","20","21"]
for h,l in zip(ls,num):
    extract('dishwasher',h,1800,1800,10,l)

In [47]:
app = 'microwave'
path ='app_activations/'+app+'/'
for i in range(20,30): #853
    temp = np.load(path+'bert_predict_h'+str(18)+'/dw'+str(i)+'_prediction.npy')
    plt.figure(figsize=(10,2))
    plt.plot(temp)